****Preprocessing

In [1]:
import torch
import torchvision
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.linear_model import LinearRegression


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#*** PREPROCESSING ***


idx_r = 300000  #limit to 300.000 to avoid RAM crash
df_train = pd.read_csv("/content/drive/MyDrive/Lithuanian_Challenge/train_data.csv")[:idx_r]
#df_test = pd.read_csv("/content/drive/MyDrive/Lithuanian_Challenge/test_data.csv")

def preprocess(df, test=False):
  df['month'] = pd.DatetimeIndex(df['Date']).month
  df['DayInMonth'] = pd.DatetimeIndex(df['Date']).day
  #df['PartOfMonth1'] = (df['DayInMonth']<=10).astype(int)
  #df['PartOfMonth2'] = ((df['DayInMonth']>10) & (df['DayInMonth']<=20)).astype(int)
  #df['PartOfMonth3'] = ((df['DayInMonth']>20) & (df['DayInMonth']<=31)).astype(int)
  #del df["DayInMonth"]
  df = df.dropna(subset=['Sales']) # drop all rows where sales = nan
  df = pd.get_dummies(data=df, columns=['Store','AssortmentType', 'StoreType', 'StateHoliday', "month", "DayOfWeek"])
  del df["Date"]
  #del df["Store"]

  Y = ""
  if test == False:
    Y = df["Sales"].values
    del df["Sales"]
  X = df.values

  return X, Y, df

X, Y, df_train = preprocess(df_train)

X_test, Y_test = X[-30000:], Y[-30000:] #30.000 test samples
X, Y = X[:-30000], Y[:-30000]  



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


***Regression Analysis

In [ ]:
import statsmodels.api as sm

x_ = sm.add_constant(df_train)
results = sm.OLS(Y,x_).fit()
results.summary()  

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


*** Support Vector Machine

In [ ]:
from sklearn import svm
from sklearn.metrics import mean_squared_error

idx = len(df)

model = svm.SVC()
model.fit(df[:idx], y_[:idx])

In [ ]:
y_pred = model.predict(df[1200:1210])
err = mean_squared_error(y_[1200:1210], y_pred, multioutput='raw_values', squared=False) # returns set of errros

In [ ]:
print("Absolute Error: ", err)

Absolute Error:  [1226.56834298]


****CNN

In [ ]:
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout


from keras import backend as K
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(float(y_pred) - float(y_true)))) 

X = X.reshape((X.shape[0], X.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

epochs = 10
batch = 256
lr = 0.001
adam = optimizers.Adam(lr)

model_cnn = Sequential()
model_cnn.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dropout(0.1))
model_cnn.add(Dense(512, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(loss=rmse, optimizer=adam)
model_cnn.summary()


cnn_history = model_cnn.fit(X, Y, epochs=epochs, validation_data=(X_test, Y_test), verbose=1) #validation_data=(X_test, Y_test)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1146, 256)         768       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 573, 256)          0         
_________________________________________________________________
flatten (Flatten)            (None, 146688)            0         
_________________________________________________________________
dropout (Dropout)            (None, 146688)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               75104768  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 75,106,049
Trainable params: 75,106,049
Non-trainable params: 0
____________________________________________